# Generate ET cell output tables from the proofread table and add putative cell type information

In [1]:
import pandas as pd
import numpy as np
import pcg_skel
import tqdm
from meshparty import meshwork
from caveclient import CAVEclient
import datetime
import os

client = CAVEclient('minnie65_phase3_v1')

pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [2]:
# MAIN
#now = datetime.datetime.utcnow()
now = client.materialize.get_timestamp()

In [3]:
# MAIN
skeldir = 'skeletons'

In [4]:
# MAIN
# Defining the HVA/VISp line:

xz0 = [237415, 26308]
xz1 = [286783, 8960]

x0 = xz0[0]
x1 = xz1[0]
z0 = xz0[1]
z1 = xz1[1]

def soma_in_hva(pt):
    ptz = pt[2]
    ptx = pt[0]
    x_thresh = x1 + (ptz-z1) * (x0-x1) / (z0-z1)
    return ptx > x_thresh

def classify_soma(pt):
    if np.any(np.isnan(pt)):
        return np.nan
        
    if soma_in_hva(pt):
        return 'hva'
    else:
        return 'v1'

In [ ]:
# OPTIONAL
#before materialization
et_df = client.materialize.live_live_query(
    "l5et_column",
    timestamp="now",
    joins=[["l5et_column", "target_id", "nucleus_detection_v0", "id"]],
    suffixes={'l5et_column': '', 'nucleus_detection_v0': '_ref'},
    metadata=False,
)

In [5]:
# MAIN
#after materialization
et_table = 'l5et_column'
et_df = client.materialize.query_table(et_table,timestamp = now)

In [14]:
# MAIN
# Build the skeletons
nrns = {}

for _, row in tqdm.tqdm(et_df.iterrows()):

    #print(row)
    if os.path.exists(f"{skeldir}/{row['pt_root_id']}.h5"):
        nrns[row["pt_root_id"]] = meshwork.load_meshwork(f"{skeldir}/{row['pt_root_id']}.h5")
   
    else:
    
        nrns[row["pt_root_id"]] = pcg_skel.coord_space_meshwork(
            row["pt_root_id"],
            client=client,
            root_point=row["pt_position"],
            root_point_resolution=[4, 4, 40],
            collapse_soma=True,
            synapses="all",
            synapse_table=client.info.get_datastack_info().get("synapse_table"),
            timestamp = now,
        )

        nrns[row["pt_root_id"]].save_meshwork(f"{skeldir}/{row['pt_root_id']}.h5")        
        
# Get the axons
for rid, nrn in nrns.items():
    is_axon = meshwork.algorithms.split_axon_by_annotation(
        nrn,
        'pre_syn',
        'post_syn',
        return_quality=False
    )
    nrn.anno.add_annotations('is_axon', is_axon, mask=True)

39it [06:28,  9.95s/it]


In [ ]:
# OPTIONAL
#Filter for presynaptic outputs on ET cell axons and concatenate into one dataframe:

pre_dfs = []
for rid in et_df["pt_root_id"]:
    df = nrns[rid].anno.pre_syn.filter_query(
            nrns[rid].anno.is_axon.mesh_mask
    ).df
    df.attrs = {}
    pre_dfs.append(df)

et_pre_df = pd.concat(pre_dfs, ignore_index=True)    

et_pre_df['pre_pt_root_id'] = client.chunkedgraph.get_roots(et_pre_df['pre_pt_supervoxel_id'], timestamp=now)

In [15]:
# MAIN
#Filter for presynaptic outputs on ET cell axons and concatenate into one dataframe, adding synapse distance:

pre_dfs = []
for rid in et_df["pt_root_id"]:
    syn_filt = nrns[rid].anno.pre_syn.filter_query(
            nrns[rid].anno.is_axon.mesh_mask
    )
    df = syn_filt.df
    df['dist_to_root'] = nrns[rid].distance_to_root(syn_filt.mesh_index)
    df['distance_rank'] = df['dist_to_root'].rank()
    df.attrs = {}
    pre_dfs.append(df)

et_pre_df= pd.concat(pre_dfs, ignore_index=True)
et_pre_df['pre_pt_root_id'] = client.chunkedgraph.get_roots(et_pre_df['pre_pt_supervoxel_id'], timestamp=now)
et_pre_df['post_pt_root_id'] = client.chunkedgraph.get_roots(et_pre_df['post_pt_supervoxel_id'], timestamp=now).astype('int')


In [16]:
# MAIN
#Add nucleus ID to dataframe

et_df_new = et_df.rename(columns={"pt_root_id": "pre_pt_root_id"})

et_pre_df = (
    et_pre_df.merge(
        et_df_new[["pre_pt_root_id", "id_ref"]].rename(
            columns={"id_ref": "pre_nucleus_id"}
        ),
        on="pre_pt_root_id",
        how="left",
    )
)

In [ ]:
# OPTIONAL

preLIST = et_pre_df.pre_pt_root_id.unique()
maxLIST = []

for pre in preLIST:
    res = [pre,max(et_pre_df[(et_pre_df['pre_pt_root_id'] == pre)].distance_rank)]
    maxLIST.append(res)
  
maxLIST

In [17]:
# MAIN
# Get single soma root ids and add cell types

soma_df = client.materialize.query_table(
    "nucleus_neuron_svm", filter_equal_dict={"cell_type": "neuron"}
)


# Add number of post_synaptic soma on a segment ID
soma_df['count_soma'] = soma_df.groupby('pt_root_id').transform('count')['id']
num_soma_df = soma_df.drop_duplicates(subset='pt_root_id')[['pt_root_id', 'count_soma']].rename(
    columns={"count_soma": "num_soma"})
num_soma_df = num_soma_df.drop_duplicates(subset="pt_root_id", keep='first')


def number_of_soma(row):               
    if pd.isna(row['num_soma']) == True:
          return 0    
    else:
          return row['num_soma']  
num_soma_df['num_soma'] = num_soma_df.apply(number_of_soma, axis=1)


# Remove all duplicates
soma_df = soma_df.drop_duplicates(subset="pt_root_id", keep='first')

# Download all the other tables we want to pull info from
NEURD_df = client.materialize.query_table("baylor_e_i_model_v1").drop_duplicates('pt_root_id', keep=False)

metamodel_df = client.materialize.query_table(
    "aibs_soma_nuc_metamodel_preds_v117",
    filter_equal_dict={"classification_system": "aibs_neuronal"},
).drop_duplicates('pt_root_id', keep=False)

mtypes_model_df = client.materialize.query_table(
    "aibs_soma_nuc_exc_mtype_preds_v117",
    filter_equal_dict={"classification_system": "aibs_coarse_excitatory"},
).drop_duplicates('pt_root_id', keep=False)

column_mtypes_df = client.materialize.query_table(
    "allen_column_mtypes_v1",
    filter_equal_dict={"classification_system": "excitatory"},
).drop_duplicates('pt_root_id', keep=False)

motif_df = client.materialize.query_table("connectivity_groups_v507").drop_duplicates('pt_root_id', keep=False)


# Enrich soma_df with all this info
soma_df = (
    soma_df.merge(
        NEURD_df[["pt_root_id", "cell_type"]].rename(
            columns={"cell_type": "NEURD_class"}
        ),
        on="pt_root_id",
        how="left",
    )
    .merge(
        metamodel_df[["pt_root_id", "cell_type"]].rename(
            columns={"cell_type": "metamodel_cell_type"}
        ),
        on="pt_root_id",
        how="left",
    )
    .merge(
        mtypes_model_df[["pt_root_id", "cell_type"]].rename(
            columns={"cell_type": "mtypes_model_cell_type"}
        ),
        on="pt_root_id",
        how="left",
    )
    .merge(
       column_mtypes_df[["pt_root_id", "cell_type"]].rename(
            columns={"cell_type": "mtypes_column"}
        ),
        on="pt_root_id",
        how="left",
    )
    .merge(
       motif_df[["pt_root_id", "cell_type"]].rename(
            columns={"cell_type": "motif_group"}
        ),
        on="pt_root_id",
        how="left",
    )
)


Table Owner Notice on nucleus_neuron_svm: Please cite https://doi.org/10.1101/2022.07.20.499976 when using this table.


In [18]:
# MAIN
#Add class labels to soma_df
def standard_class_metamodel(row):
        
    if row['metamodel_cell_type'] == 'MC':
          return 'inhibitory'
 
    if row['metamodel_cell_type'] == 'BC':
          return 'inhibitory'
          
    if row['metamodel_cell_type'] == 'NGC':
          return 'inhibitory'
        
    if row['metamodel_cell_type'] == 'BPC':
          return 'inhibitory'
     
    if row['metamodel_cell_type'] == 'none':
          return None
               
    if pd.isna(row['metamodel_cell_type']) == True:
          return None
        
    else:
          return 'excitatory' 

soma_df['metamodel_class'] = soma_df.apply(standard_class_metamodel, axis=1)

In [19]:
#MAIN
#standardize class labels
def standard_class_NEURD(row):
        
    if pd.isna(row['NEURD_class']) == True:
          return None
    else:
          return row['NEURD_class']  

soma_df['NEURD_class'] = soma_df.apply(standard_class_NEURD, axis=1)

In [20]:
#MAIN
#standardize sub_class labels
def standard_subclass_metamodel(row):
        
    if row['metamodel_cell_type'] == '6P-IT':
          return '6P'

    if row['metamodel_cell_type'] == '6P-CT':
          return '6P'
    
    if pd.isna(row['metamodel_cell_type']) == True:
          return None
    else:
          return row['metamodel_cell_type']  

soma_df['metamodel_cell_type'] = soma_df.apply(standard_subclass_metamodel, axis=1)

In [21]:
#MAIN
#standardize sub_class labels
def standard_subclass_mytpes(row):
        
    if row['mtypes_column'] == 'L3c':
          return '23P'
    
    if row['mtypes_column'] == 'L5ET':
          return '5P-ET'

    if row['mtypes_column'] == 'L2b':
          return '23P'
        
    if row['mtypes_column'] == 'L6a':
          return '6P'
  
    if row['mtypes_column'] == 'L4c':
          return '4P'
        
    if row['mtypes_column'] == 'L6c':
          return '6P'
        
    if row['mtypes_column'] == 'L6CT':
          return '6P'
        
    if row['mtypes_column'] == 'L6b':
          return '6P'
        
    if row['mtypes_column'] == 'L4a':
          return '4P'
        
    if row['mtypes_column'] == 'L2a':
          return '23P'
        
    if row['mtypes_column'] == 'L3b':
          return '23P'
        
    if row['mtypes_column'] == 'L3a':
          return '23P'

    if row['mtypes_column'] == 'L5b':
          return '5P-IT'

    if row['mtypes_column'] == 'L4b':
          return '4P'

    if row['mtypes_column'] == 'L5a':
          return '5P-IT'

    if row['mtypes_column'] == 'L5NP':
          return '5P-NP'

    if row['mtypes_column'] == 'L6wm':
          return '6P'

    if pd.isna(row['mtypes_column']) == True:
          return None
    else:
          return row['mtypes_column']  

soma_df['mtypes_column'] = soma_df.apply(standard_subclass_mytpes, axis=1)


In [22]:
#MAIN
#standardize sub_class labels
def standard_subclass_mytpes_model(row):
        
    if row['mtypes_model_cell_type'] == 'L3c':
          return '23P'
    
    if row['mtypes_model_cell_type'] == 'L5ET':
          return '5P-ET'

    if row['mtypes_model_cell_type'] == 'L2b':
          return '23P'
        
    if row['mtypes_model_cell_type'] == 'L6a':
          return '6P'
  
    if row['mtypes_model_cell_type'] == 'L4c':
          return '4P'
        
    if row['mtypes_model_cell_type'] == 'L6c':
          return '6P'
        
    if row['mtypes_model_cell_type'] == 'L6CT':
          return '6P'
        
    if row['mtypes_model_cell_type'] == 'L6b':
          return '6P'
        
    if row['mtypes_model_cell_type'] == 'L4a':
          return '4P'
        
    if row['mtypes_model_cell_type'] == 'L2a':
          return '23P'
        
    if row['mtypes_model_cell_type'] == 'L3b':
          return '23P'
        
    if row['mtypes_model_cell_type'] == 'L3a':
          return '23P'

    if row['mtypes_model_cell_type'] == 'L5b':
          return '5P-IT'

    if row['mtypes_model_cell_type'] == 'L4b':
          return '4P'

    if row['mtypes_model_cell_type'] == 'L5a':
          return '5P-IT'

    if row['mtypes_model_cell_type'] == 'L5NP':
          return '5P-NP'

    if row['mtypes_model_cell_type'] == 'L6wm':
          return '6P'

    if pd.isna(row['mtypes_model_cell_type']) == True:
          return None
    else:
          return row['mtypes_model_cell_type']  

soma_df['mtypes_model_cell_type'] = soma_df.apply(standard_subclass_mytpes_model, axis=1)


In [23]:
# MAIN
#Merge all this info from cell types into the synapse dataframe, as well as add area locations.

#merge presynaptic nucleous ID
synapse_table = et_pre_df.merge(
    soma_df[
        ["id", "pt_root_id", "pt_position",  "NEURD_class", "metamodel_class", "metamodel_cell_type","mtypes_model_cell_type","mtypes_column","motif_group"]
    ].rename(columns={"pt_position": "post_soma_pt"}).rename(columns={"id": "post_nucleus_id"}),
    left_on="post_pt_root_id",
    right_on="pt_root_id",
    how="left",
).drop(columns="pt_root_id")

synapse_table["post_soma_area"] = synapse_table['post_soma_pt'].apply(classify_soma)

synapse_table = synapse_table.merge(
    et_df[['pt_root_id', 'pt_position']].rename(columns={'pt_position': 'pre_soma_pt'}),
    left_on='pre_pt_root_id',
    right_on='pt_root_id',
    how='left',
).drop(columns='pt_root_id')

synapse_table["pre_soma_area"] = synapse_table['pre_soma_pt'].apply(classify_soma)

synapse_table = synapse_table.rename(columns={"cell_type_pred": "aibs_auto_subclass"})


# addition by Nuno
# load manual label

manual_multi_df = client.materialize.live_live_query(
    'pt_synapse_targets',
    timestamp="now",
    metadata=False,
)
#manual_multi_df = client.materialize.query_table("pt_synapse_targets")
manual_multi_df = manual_multi_df.rename(columns={"target_id": "synapse_id"})
#manual_multi_df = client.materialize.query_table("pt_synapse_targets").drop_duplicates('post_pt_root_id', keep=False)
#manual_multi_df['post_pt_root_id'] = manual_multi_df.post_pt_root_id.astype('UInt64')


#manual_multi_df = pd.read_feather('manual_pt.feather')

synapse_table = synapse_table.rename(columns={"id": "synapse_id"})


#merge manual labels

synapse_table = (
    synapse_table.merge(
        manual_multi_df[["synapse_id", "classification_system"]].rename(
            columns={"classification_system": "manual_class"}
        ),
        on='synapse_id',
        how="left",
    )
    .merge(
        manual_multi_df[["synapse_id", "cell_type"]].rename(
            columns={"cell_type": "manual_subclass"}
        ),
        on='synapse_id',
        how="left",
    )
    .merge(
        num_soma_df[["pt_root_id", "num_soma"]],
        left_on='post_pt_root_id',
        right_on='pt_root_id',
        how="left",
    ).drop(columns='pt_root_id')
)


def number_of_soma(row):               
    if pd.isna(row['num_soma']) == True:
          return 0    
    else:
          return row['num_soma']  
synapse_table['num_soma'] = synapse_table.apply(number_of_soma, axis=1)



In [24]:
#MAIN
#standardize class labels from manual
def standard_class_man(row):
        
    if row['manual_class'] == 'none':
          return None        

    if pd.isna(row['manual_class']) == True:
          return None
         
    else:
          return row['manual_class']  

synapse_table['manual_class'] = synapse_table.apply(standard_class_man, axis=1)

synapse_table = synapse_table[(synapse_table['manual_class'] != 'error')]

In [25]:
#MAIN
#standardize sub_class labels
def standard_subclass_man(row):
        
    if row['manual_subclass'] == 'multisoma':
          return None

    if row['manual_subclass'] == 'DTC':
          return 'MC'               
               
    if row['manual_subclass'] == 'none':
          return None
        
    if row['manual_subclass'] == '5P-PT':
          return '5P-ET'

    if row['manual_subclass'] == 'unclear':
          return None
                 
    if pd.isna(row['manual_subclass']) == True:
          return None
    else:
          return row['manual_subclass']  

synapse_table['manual_subclass'] = synapse_table.apply(standard_subclass_man, axis=1)

In [26]:
# MAIN
# create new column indicating which neurons are in 
# Group 1 (non-ET prefereing interneurons) and Group 2 (ET-prefering Interneurons)

#create new column 'inhibitory groups'
def create_inhibitory_groups(row):

    Group2 = [269633, 305251,269414,267006,305232,303195,271673,269585,292721,
          304990,305233,269485,269334,305046,303172, 267293,302962]
    
    if pd.isna(row['motif_group']) == True:
          return 0
    else:
    
        if row['post_nucleus_id'] in  Group2:
            return 2
        else:
            return 1

synapse_table['inhibitory_groups'] = synapse_table.apply(create_inhibitory_groups, axis=1)


synapse_table.inhibitory_groups.unique()

array([0, 1, 2])

In [27]:
# MAIN
#QC - CHECK IF THERE ARE DFERRENT MANUAL CLASS LABELS ASIGNED TO THE SAME NEURON 

for ii in synapse_table.post_pt_root_id.unique():

    if len(synapse_table[(synapse_table['post_pt_root_id'] == ii) &
                    pd.notna(synapse_table['manual_class'])].manual_class.unique()) > 1:
    #if len(synapse_table[(synapse_table['post_pt_root_id'] == ii) & (synapse_table['num_soma'] < 2) &
    #                pd.notna(synapse_table['manual_class'])].manual_class.unique()) > 1:
        print(ii)    

In [28]:
# MAIN
#QC - CHECK IF THERE ARE DFERRENT MANUAL SUBCLASS LABELS ASIGNED TO THE SAME NEURON 

for ii in synapse_table.post_pt_root_id.unique():

    if len(synapse_table[(synapse_table['post_pt_root_id'] == ii) & (synapse_table['num_soma'] < 2) 
                         & pd.notna(synapse_table['manual_subclass'])].manual_subclass.unique()) > 1:
        print(ii)
        
    if len(synapse_table[(synapse_table['post_pt_root_id'] == ii) & (synapse_table['num_soma'] < 2) 
                         & pd.notna(synapse_table['manual_subclass'])].manual_subclass.unique()) > 1:

        print(ii) 

In [29]:
# MAIN
#TRANSFER MANUAL SUBCLASS LABELS ACROSS SYNAPSES OF THE SAME NEURON

#Create df with subclass labels and only one post_pt_root_id for IDs that are single somas or orphans
manual_subclass_labels = synapse_table[(synapse_table['num_soma'] <= 1) &
                                      pd.notna(synapse_table['manual_subclass'])].drop_duplicates(subset='post_pt_root_id')

manual_subclass_labels = manual_subclass_labels[['post_pt_root_id', 'manual_subclass']] 

#Transfer the subclass labels using the merge function 
#(In an earlier version I created a new table after this point "#synapse_table_transfer")

synapse_table = synapse_table.merge(manual_subclass_labels, left_on='post_pt_root_id',
                                                      right_on='post_pt_root_id', how='left')

#Transfer the subclass labels on multisoma
def subclass_transfer(row):
   
    if pd.isna(row['manual_subclass_y']) == True:
          return row['manual_subclass_x']
   
    else:
          return row['manual_subclass_y']  

synapse_table['manual_subclass_y'] = synapse_table.apply(subclass_transfer, axis=1)

#Rename columns
synapse_table = synapse_table.rename(columns={"manual_subclass_x": "manual_subclass_original",
                                                                "manual_subclass_y": "manual_subclass"})


In [30]:
# MAIN
#TRANSFER MANUAL CLASS LABELS ACROSS SYNAPSES OF THE SAME NEURON

#Create df with subclass labels and only one post_pt_root_id for IDs that are single somas or orphans
manual_class_labels = synapse_table[(synapse_table['num_soma'] <= 1) &
                                      pd.notna(synapse_table['manual_class'])].drop_duplicates(subset='post_pt_root_id')

manual_class_labels = manual_class_labels[['post_pt_root_id', 'manual_class']] 

#Transfer the subclass labels using the merge function
synapse_table = synapse_table.merge(manual_class_labels, left_on='post_pt_root_id',
                                                      right_on='post_pt_root_id', how='left')

#Transfer multisoma labels
def class_transfer(row):
   
    if pd.isna(row['manual_class_y']) == True:
          return row['manual_class_x']
   
    else:
          return row['manual_class_y']  

synapse_table['manual_class_y'] = synapse_table.apply(class_transfer, axis=1)

#Rename columns
synapse_table = synapse_table.rename(columns={"manual_class_x": "manual_class_original",
                                              "manual_class_y": "manual_class"})


In [31]:
# MAIN
#QC - CHECK IF THE MANUAL CLASS AND SUBCLASS ARE CONSISTENT

#create new column where class is calculated from subclass
def create_class_from_subclass(row):
    if row['manual_subclass'] == '5P-NP':
          return 'excitatory'
    if row['manual_subclass'] == '5P-ET':
          return 'excitatory'
    if row['manual_subclass'] == '5P-IT':
          return 'excitatory'
    if row['manual_subclass'] == '4P':
          return 'excitatory'
    if row['manual_subclass'] == '6P':
          return 'excitatory'
    if row['manual_subclass'] == '23P':
          return 'excitatory'
    if row['manual_subclass'] == 'BC':
          return 'inhibitory'
    if row['manual_subclass'] == 'MC':
          return 'inhibitory'
    if row['manual_subclass'] == 'BPC':
          return 'inhibitory'
    else:
          return row['manual_subclass']

synapse_table['class_from_subclass'] = synapse_table.apply(create_class_from_subclass, axis=1)


def check_class_from_subclass(row):
   
    if row['manual_class'] == row['class_from_subclass']:
          return 'OK'
   
    else:
          return row['manual_subclass']  

synapse_table['check_class_from_subclass'] = synapse_table.apply(check_class_from_subclass, axis=1)

synapse_table.check_class_from_subclass.unique()


array(['OK', None], dtype=object)

In [32]:
# MAIN
#QC - CHECK DISAGREEMENT BETWEEN NEURD and metamodel LABELS

manual_check1 = synapse_table[(synapse_table['NEURD_class'] == 'excitatory') 
              & (synapse_table['metamodel_class'] == 'inhibitory') & pd.isna(synapse_table['manual_class'])].drop_duplicates(subset='post_pt_root_id')#.post_pt_root_id.unique()

print('number of unchecked disagreements where NEURD is "E" and metamodel is "I": ', len(manual_check1))


manual_check2 = synapse_table[(synapse_table['NEURD_class'] == 'inhibitory') 
              & (synapse_table['metamodel_class'] == 'excitatory') & pd.isna(synapse_table['manual_class'])].drop_duplicates(subset='post_pt_root_id')#.post_pt_root_id.unique()

print('number of unchecked disagreements where NEURD is "I" and metamodel is "E": ', len(manual_check2))


number of unchecked disagreements where NEURD is "E" and metamodel is "I":  0
number of unchecked disagreements where NEURD is "I" and metamodel is "E":  2


In [33]:
#MAIN
#QC - CHECK FOR LABELS WITH NO ENTRIES

manual_check = synapse_table[pd.isna(synapse_table['NEURD_class']) 
              & pd.isna(synapse_table['metamodel_cell_type']) & pd.isna(synapse_table['manual_class'])].drop_duplicates(subset='post_pt_root_id')#.post_pt_root_id.unique()

manual_check

,ctr_pt_position,synapse_id,post_pt_position,post_pt_root_id,post_pt_supervoxel_id,pre_pt_level2_id,pre_pt_mesh_ind,pre_pt_position,pre_pt_root_id,pre_pt_supervoxel_id,size,pre_pt_mesh_ind_filt,dist_to_root,distance_rank,pre_nucleus_id,post_nucleus_id,post_soma_pt,NEURD_class,metamodel_class,metamodel_cell_type,mtypes_model_cell_type,mtypes_column,motif_group,post_soma_area,pre_soma_pt,pre_soma_area,manual_class_original,manual_subclass_original,num_soma,inhibitory_groups,manual_subclass,manual_class,class_from_subclass,check_class_from_subclass
14,"[145454, 214834, 25356]",101679816,"[145462, 214886, 25356]",864691136385641232,84815022639078738,1.568726e+17,535,"[145466, 214762, 25361]",864691135781927760,84815022639079666,4704,535,320399.049683,126.0,267008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[169216, 196416, 20820]",v1,None,None,0.0,0,None,None,None,OK
18,"[139460, 210478, 26091]",98701527,"[139506, 210490, 26100]",864691136682744174,83969979301044742,1.560276e+17,180,"[139480, 210492, 26090]",864691135781927760,83969979301032519,17488,180,390403.640717,144.0,267008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[169216, 196416, 20820]",v1,None,None,0.0,0,None,None,None,OK
25,"[144268, 213053, 26340]",102222880,"[144270, 213064, 26340]",864691133495821357,84603641662856196,1.566612e+17,422,"[144318, 212998, 26342]",864691135781927760,84603641662851097,19844,422,359538.942688,135.0,267008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[169216, 196416, 20820]",v1,None,None,0.0,0,None,None,None,OK
33,"[151428, 214222, 24005]",120702860,"[151476, 214244, 24009]",864691136102143485,85589009904451309,1.576466e+17,1009,"[151470, 214258, 24003]",864691135781927760,85589009904447441,3872,1009,256125.170166,102.0,267008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[169216, 196416, 20820]",v1,None,None,0.0,0,None,None,None,OK
35,"[127696, 210480, 17873]",75905628,"[127724, 210532, 17874]",864691132324282180,82351497111182425,1.544091e+17,3,"[127690, 210448, 17866]",864691135781927760,82351497111174576,8884,3,334330.370110,128.0,267008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[169216, 196416, 20820]",v1,None,None,0.0,0,None,None,None,OK
36,"[140945, 211385, 26273]",98732369,"[140916, 211302, 26269]",864691136172059405,84181223039242869,1.561684e+17,236,"[140832, 211332, 26269]",864691135781927760,84110854295062374,23756,236,380196.837250,143.0,267008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[169216, 196416, 20820]",v1,None,None,0.0,0,None,None,None,OK
37,"[178520, 210978, 21501]",160916290,"[178542, 210938, 21509]",864691134469948667,89318072040595012,1.613757e+17,7957,"[178590, 210904, 21494]",864691135781927760,89318071974092189,10120,7957,254090.430511,101.0,267008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[169216, 196416, 20820]",v1,None,None,0.0,0,None,None,None,OK
40,"[145176, 213716, 24928]",103866786,"[145164, 213698, 24923]",864691134753726326,84744447669437272,1.568021e+17,484,"[145126, 213738, 24927]",864691135781927760,84744516388948200,10896,484,303324.282349,115.0,267008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[169216, 196416, 20820]",v1,None,None,0.0,0,None,None,None,OK
76,"[182882, 200102, 21644]",165999378,"[182826, 200154, 21643]",864691134552867508,89949947629332297,1.620075e+17,8619,"[182920, 200112, 21645]",864691135781927760,89949947629336832,5092,8619,306731.298584,116.5,267008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[169216, 196416, 20820]",v1,None,None,0.0,0,None,None,None,OK
77,"[144054, 212983, 26369]",102222628,"[144062, 212954, 26374]",864691133403142052,84603641662882259,1.566612e+17,423,"[143950, 212940, 26365]",864691135781927760,84603641662874848,9204,423,364061.639465,138.0,267008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[169216, 196416, 20820]",v1,None,None,0.0,0,None,None,None,OK


In [34]:
#MAIN
#INTEGRATE CLASS LABELS BETWEEN MANUAL AND AUTOMATED LABELS

#generate new consensus column
synapse_table['consensus_class'] = synapse_table['manual_class']

#When there isn't manual label add aibs_v2 label

def integrate_class(row):
    if row['consensus_class'] == None:
          return row['metamodel_class']
    
    else:
          return row['consensus_class']  

synapse_table['consensus_class'] = synapse_table.apply(integrate_class, axis=1)  

In [35]:
#MAIN
#INTEGRATE SUBCLASS LABELS BETWEEN MANUAL AND AUTOMATED LABELS

#generate new consensus column
synapse_table['consensus_subclass'] = synapse_table['manual_subclass']

#When there isn't manual label add aibs_v2 label

def integrate_subclass(row):
    if row['consensus_subclass'] == None:
          return row['metamodel_cell_type']
    if row['consensus_subclass'] == 'inhibitory':
          return None      
    
    else:
          return row['consensus_subclass']  

synapse_table['consensus_subclass'] = synapse_table.apply(integrate_subclass, axis=1)  

In [36]:
#MAIN
#QC - CHECK IF THE INTEGRATED CLASS AND SUBCLASS ARE CONSISTENT

#remove previous columns
synapse_table = synapse_table.drop(['class_from_subclass', 'check_class_from_subclass'], axis=1)
#synapse_table = synapse_table.drop(['class_from_subclass'], axis=1)


#create new column where class is calculated from subclass
def create_class_from_subclass(row):
    if row['consensus_subclass'] == '5P-NP':
          return 'excitatory'
    if row['consensus_subclass'] == '5P-ET':
          return 'excitatory'
    if row['consensus_subclass'] == '5P-IT':
          return 'excitatory'
    if row['consensus_subclass'] == '4P':
          return 'excitatory'
    if row['consensus_subclass'] == '6P':
          return 'excitatory'
    if row['consensus_subclass'] == '6P-IT':
          return 'excitatory'
    if row['consensus_subclass'] == '6P-CT':
          return 'excitatory'
    if row['consensus_subclass'] == '23P':
          return 'excitatory'
    if row['consensus_subclass'] == 'BC':
          return 'inhibitory'
    if row['consensus_subclass'] == 'MC':
          return 'inhibitory'
    if row['consensus_subclass'] == 'NGC':
          return 'inhibitory'
    if row['consensus_subclass'] == 'BPC':
          return 'inhibitory'
    if row['consensus_subclass'] == 'inhibitory':
          return 'unknown'        
        
    else:
          return row['consensus_subclass']

synapse_table['class_from_subclass'] = synapse_table.apply(create_class_from_subclass, axis=1)


def check_class_from_subclass(row):
   
    if row['consensus_class'] == row['class_from_subclass']:
          return 'OK'
   
    else:
          return row['consensus_subclass']  

synapse_table['check_class_from_subclass'] = synapse_table.apply(check_class_from_subclass, axis=1)

synapse_table.check_class_from_subclass.unique()


array(['OK', nan, None, '6P', '5P-IT', '4P', 'MC', '5P-ET'], dtype=object)

In [ ]:
synapse_table[synapse_table['check_class_from_subclass'] == '6P-IT']

In [ ]:
synapse_table.manual_subclass_original.unique()

In [ ]:
#CREATE NEUROGLANCER LINK


manual_check = synapse_table[(synapse_table['synapse_id']== 177039637)].drop_duplicates(subset='post_pt_root_id')
#manual_check = synapse_table[(synapse_table['manual_eiaibs_subclass']== '5P-PT') & (synapse_table['pre_pt_root_id']==864691135293076662)].drop_duplicates(subset='post_pt_root_id')
#manual_check = manual_check.drop_duplicates(subset='post_pt_root_id')


from nglui import statebuilder

img, seg = statebuilder.from_client(client)

pt_map = statebuilder.PointMapper('post_pt_position', linked_segmentation_column='post_pt_root_id')
anno = statebuilder.AnnotationLayerConfig('post_pt_position', mapping_rules=pt_map, linked_segmentation_layer=seg.name,
                                          tags=['single_spine','dendrite', 'error', 'has_soma'])
sb = statebuilder.StateBuilder([img, seg, anno], client=client)

#here is where you add the dataframe
sb.render_state(manual_check[['post_pt_root_id','post_pt_position']], return_as='html')

#[id, x,y,z]

In [37]:
# MAIN
#SAVE AND READ

#remove columns before saving
#synapse_table = synapse_table.drop(['class_from_subclass', 'check_class_from_subclass'], axis=1)


#save et_pre_ct_df
synapse_table = synapse_table[synapse_table['distance_rank'] < 101]
synapse_table.reset_index(drop=True).to_feather("ET_Column_synapse_table.feather")

#READ
#et_pre_ct_df = pd.read_feather('ET_Column_syn_df_NC.feather')


# EXTRAS TO BE DELETED

In [ ]:
# OPTIONAL

#ANALYSIS - FIGURE 2.g
#percentage of Inhibitory group 2 targets


#number of synapses with Inhibitory group 1 targets
group1 = len(synapse_table[(synapse_table['inhibitory_groups'] ==  1)])

#number of synapses with Inhibitory group 2 targets
group2 = len(synapse_table[(synapse_table['inhibitory_groups'] ==  2)])


import matplotlib.pyplot as plt
import numpy as np
plt.style.use('_mpl-gallery')

# make data:
np.random.seed(3)
x = ['Group 1', 'Group 2']
y = [group1, group2]

# plot
fig, ax = plt.subplots()

ax.bar(x, y, width=1, edgecolor="white", linewidth=0.5)

#ax.set(xlim=(0, 8), xticks=np.arange(1, 8),
#       ylim=(0, 8), yticks=np.arange(1, 8))

fig.savefig('Column_inhibitory_groups_input.eps', bbox_inches='tight')

plt.show()







In [ ]:
# MAIN

#ANALYSIS - FIGURE 2.f
#percentage of Inhibitory group 2 targets
#after materialization


column_table = 'allen_column_mtypes_v1'
column_table_df = client.materialize.query_table(column_table,timestamp = now)


all_ids = column_table_df.id_ref.values


column_table_I_df = column_table_df[(column_table_df['classification_system'] ==  'inhibitory')]





In [ ]:
# MAIN
# crop dataframe to incliude ony synapses with column neurons

def create_column_neurons(row):

#    GroupET = column_table_df[column_table_df['cell_type'] == 'L5ET'].target_id.values
#    
#    Group2 = [269633, 305251,269414,267006,305232,303195,271673,269585,292721,
#          304990,305233,269485,269334,305046,303172, 267293,302962]
    
    if row['post_nucleus_id'] in GroupET:
    
        if row['pre_nucleus_id'] in  Group2:
            return 2
        else:
            return 1

column_table_I_pre_df['inhibitory_groups'] = column_table_I_pre_df.apply(create_inhibitory_groups, axis=1)


column_table_I_pre_df.inhibitory_groups.unique()

In [ ]:
column_table_I_df = column_table_df[(column_table_df['classification_system'] ==  'inhibitory')]

In [ ]:
column_table_df.classification_system.unique()

In [ ]:
column_table_I_df

In [ ]:
# MAIN
# Build the skeletons
nrns = {}

for _, row in tqdm.tqdm(column_table_I_df.iterrows()):

    #print(row)
    if os.path.exists(f"{skeldir}/{row['pt_root_id']}.h5"):
        nrns[row["pt_root_id"]] = meshwork.load_meshwork(f"{skeldir}/{row['pt_root_id']}.h5")
   
    else:
    
        nrns[row["pt_root_id"]] = pcg_skel.coord_space_meshwork(
            row["pt_root_id"],
            client=client,
            root_point=row["pt_position"],
            root_point_resolution=[4, 4, 40],
            collapse_soma=True,
            synapses="all",
            synapse_table=client.info.get_datastack_info().get("synapse_table"),
            timestamp = now,
        )

        nrns[row["pt_root_id"]].save_meshwork(f"{skeldir}/{row['pt_root_id']}.h5")        
        
# Get the axons
for rid, nrn in nrns.items():
    is_axon = meshwork.algorithms.split_axon_by_annotation(
        nrn,
        'pre_syn',
        'post_syn',
        return_quality=False
    )
    nrn.anno.add_annotations('is_axon', is_axon, mask=True)

In [ ]:
# OPTIONAL
#Filter for presynaptic outputs on ET cell axons and concatenate into one dataframe:

pre_dfs = []
for rid in column_table_I_df["pt_root_id"]:
    df = nrns[rid].anno.pre_syn.filter_query(
            nrns[rid].anno.is_axon.mesh_mask
    ).df
    df.attrs = {}
    pre_dfs.append(df)

column_table_I_pre_df = pd.concat(pre_dfs, ignore_index=True)    

column_table_I_pre_df['pre_pt_root_id'] = client.chunkedgraph.get_roots(column_table_I_pre_df['pre_pt_supervoxel_id'], timestamp=now)

In [ ]:
#merge postynaptic nucleous ID
column_table_I_pre_df = column_table_I_pre_df.merge(
    soma_df[
        ["id", "pt_root_id", "pt_position"]
    ].rename(columns={"pt_position": "post_soma_pt"}).rename(columns={"id": "post_nucleus_id"}),
    left_on="post_pt_root_id",
    right_on="pt_root_id",
    how="left",
).drop(columns="pt_root_id")

In [ ]:
# MAIN
#Consider only synapses with column neurons

def target_column_neurons(row):
    
    if row['post_nucleus_id'] in all_ids:
    
        return 1
    else:
        return 0

column_table_I_pre_df['column_neuron'] = column_table_I_pre_df.apply(target_column_neurons, axis=1)

In [ ]:
column_table_I_pre_df = column_table_I_pre_df[(column_table_I_pre_df['classification_system'] ==  'inhibitory')]

In [ ]:
column_table_I_pre_df.head

In [ ]:
# MAIN
#Consider only synapses with column neurons

def target_column_neurons(row):

#    GroupET = column_table_df[column_table_df['cell_type'] == 'L5ET'].target_id.values
#    
#    Group2 = [269633, 305251,269414,267006,305232,303195,271673,269585,292721,
          304990,305233,269485,269334,305046,303172, 267293,302962]
    
    if row['post_nucleus_id'] in all_ids:
    
        return 1
    else:
        return 0

column_table_I_pre_df['column_neuron'] = column_table_I_pre_df.apply(target_column_neurons, axis=1)


column_table_I_pre_df.inhibitory_groups.unique()

In [ ]:
column_table_I_pre_df.head()

In [ ]:
column_table_I_pre_df = column_table_I_pre_df.merge(
    soma_df[
        ["id","pt_root_id"]
    ].rename(columns={"id": "pre_nucleus_id"}),
    left_on="pre_pt_root_id",
    right_on="pt_root_id",
    how="left",
).drop(columns="pt_root_id")

In [ ]:
column_table_I_pre_df.cell_type_pred.unique()

In [ ]:
soma_df.head()

In [ ]:
# MAIN
# create new column indicating which neurons are in 
# Group 1 (non-ET prefereing interneurons) and Group 2 (ET-prefering Interneurons)

#create new column 'inhibitory groups'
def create_inhibitory_groups(row):

    GroupET = column_table_df[column_table_df['cell_type'] == 'L5ET'].target_id.values
    
    Group2 = [269633, 305251,269414,267006,305232,303195,271673,269585,292721,
          304990,305233,269485,269334,305046,303172, 267293,302962]
    
    if row['post_nucleus_id'] in GroupET:
    
        if row['pre_nucleus_id'] in  Group2:
            return 2
        else:
            return 1

column_table_I_pre_df['inhibitory_groups'] = column_table_I_pre_df.apply(create_inhibitory_groups, axis=1)


column_table_I_pre_df.inhibitory_groups.unique()

In [ ]:
# OPTIONAL

#ANALYSIS - FIGURE 2.g
#percentage of Inhibitory group 2 targets


#number of synapses with Inhibitory group 1 targets
group1 = len(synapse_table[(synapse_table['inhibitory_groups'] ==  1)])

#number of synapses with Inhibitory group 2 targets
group2 = len(synapse_table[(synapse_table['inhibitory_groups'] ==  2)])


import matplotlib.pyplot as plt
import numpy as np
plt.style.use('_mpl-gallery')

# make data:
np.random.seed(3)
x = ['Group 1', 'Group 2']
y = [group1, group2]

# plot
fig, ax = plt.subplots()

ax.bar(x, y, width=1, edgecolor="white", linewidth=0.5)

#ax.set(xlim=(0, 8), xticks=np.arange(1, 8),
#       ylim=(0, 8), yticks=np.arange(1, 8))

fig.savefig('Column_inhibitory_groups_input.eps', bbox_inches='tight')

plt.show()






In [ ]:
column_table_df[column_table_df['cell_type'] == 'L5ET'].target_id.values


In [ ]:
column_table_df.cell_type.unique()

In [ ]:
column_table_I_pre_df = column_table_I_pre_df.drop(columns="inhibitory_groups")

In [ ]:
column_table_I_pre_df.head()

In [ ]:
synapse_table[(synapse_table['inhibitory_groups'] ==  1)]


In [ ]:
len(synapse_table[(synapse_table['inhibitory_groups'] ==  2)].post_nucleus_id.unique())

In [ ]:
group1

In [ ]:
#MAIN
#ANALYSIS GENERAL
##ANALYSIS - FIGURE 4.a
#MAKE STATS DATAFRAME - All synapses and connections

np.seterr(divide='ignore', invalid='ignore')

percentage = []
values = []

#IDs of presynaptic PT neurons
pre_soma_IDs = All_neurons


for ii,pre_soma_ID in enumerate(pre_soma_IDs):        
        
   
    stat_values={

                    'ID': pre_soma_ID, 
                      #SYNAPSES
                    
                    'all_syn_total': len(synapse_table[ (synapse_table['pre_pt_root_id'] == pre_soma_ID)]),
                    'all_e_syn#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_class'] == 'excitatory')]),
                    'all_i_syn#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_class'] == 'inhibitory')]),
        
                    'all_Undetermined_class_syn#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     ((synapse_table['manual_eiaibs_class'] == 'Unsure') |
                                      pd.isnull(synapse_table['manual_eiaibs_class']))]),
                    
        
                    'all_23P_syn#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == '23P')]),
        
                    'all_4P_syn#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == '4P')]),
                    'all_5P-PT_syn#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == '5P-PT')]),
                    'all_5P-NP_syn#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == '5P-NP')]), 
                    'all_5P-IT_syn#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == '5P-IT')]), 
                
                    'all_6P_syn#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == '6P')]),
        
                    'all_BC_syn#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == 'BC')]),
        
                    'all_MC_syn#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == 'MC')]),
        
                    'all_BPC_syn#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == 'BPC')]),
        
                    'all_NGC_syn#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == 'NGC')]),
        
                    'all_Undetermined_subclass_syn#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     ((synapse_table['manual_eiaibs_subclass'] == 'Unsure') |
                                      pd.isnull(synapse_table['manual_eiaibs_subclass']))]),
                    
        
        #CONNECTIONS
                    'all_con_total': len(synapse_table[ (synapse_table['pre_pt_root_id'] == pre_soma_ID)]
                                         ['post_pt_root_id'].unique()),
                    
                    'all_e_con#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_class'] == 'excitatory')]
                                      ['post_pt_root_id'].unique()),
                    
                    'all_i_con#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_class'] == 'inhibitory')]
                                      ['post_pt_root_id'].unique()),
        
                    'all_Undetermined_class_con#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     ((synapse_table['manual_eiaibs_class'] == 'Unsure') |
                                      pd.isnull(synapse_table['manual_eiaibs_class']))]['post_pt_root_id'].unique()),                      
        
                    'all_23P_con#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == '23P')]['post_pt_root_id'].unique()),
        
                    'all_4P_con#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == '4P')]['post_pt_root_id'].unique()),
                    
                    'all_5P-PT_con#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == '5P-PT')]['post_pt_root_id'].unique()),
                    
                    'all_5P-NP_con#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == '5P-NP')]['post_pt_root_id'].unique()), 
 
                    'all_5P-IT_con#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == '5P-IT')]['post_pt_root_id'].unique()),
                    
                    'all_6P_con#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == '6P')]['post_pt_root_id'].unique()),
                                    
                    'all_BC_con#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == 'BC')]['post_pt_root_id'].unique()),
                     
                    'all_MC_con#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == 'MC')]['post_pt_root_id'].unique()),
       
                    'all_BPC_con#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == 'BPC')]['post_pt_root_id'].unique()),
                    
                    'all_NGC_con#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     (synapse_table['manual_eiaibs_subclass'] == 'NGC')]['post_pt_root_id'].unique()),
        
                    'all_Undetermined_subclass_con#': len(synapse_table[(synapse_table['pre_pt_root_id'] == pre_soma_ID) &
                                     ((synapse_table['manual_eiaibs_subclass'] == 'Unsure') |
                                      pd.isnull(synapse_table['manual_eiaibs_subclass']))]['post_pt_root_id'].unique()),
                        
        
    }
    values.append(stat_values)
    
    
    stat_percentage={

                    'ID': pre_soma_ID, 
                      
                    'i_syn': stat_values['all_i_syn#'] / stat_values['all_syn_total'],
                    'e_syn': stat_values['all_e_syn#'] / stat_values['all_syn_total'],
                    'Undetermined_class_syn': stat_values['all_Undetermined_class_syn#'] / stat_values['all_syn_total'],
        
                    '23P_syn': stat_values['all_23P_syn#'] / (stat_values['all_syn_total'] - stat_values['all_Undetermined_subclass_syn#']),
                    '4P_syn': stat_values['all_4P_syn#'] / (stat_values['all_syn_total'] - stat_values['all_Undetermined_subclass_syn#']),
                    '5P-PT_syn': stat_values['all_5P-PT_syn#'] / (stat_values['all_syn_total'] - stat_values['all_Undetermined_subclass_syn#']),
                    '5P-IT_syn': stat_values['all_5P-IT_syn#'] / (stat_values['all_syn_total'] - stat_values['all_Undetermined_subclass_syn#']),
                    '5P-NP_syn': stat_values['all_5P-NP_syn#'] / (stat_values['all_syn_total'] - stat_values['all_Undetermined_subclass_syn#']),
                    '6P_syn': stat_values['all_6P_syn#'] / (stat_values['all_syn_total'] - stat_values['all_Undetermined_subclass_syn#']),
                    'BC_syn': stat_values['all_BC_syn#'] / (stat_values['all_syn_total'] - stat_values['all_Undetermined_subclass_syn#']),
                    'MC_syn': stat_values['all_MC_syn#'] / (stat_values['all_syn_total'] - stat_values['all_Undetermined_subclass_syn#']),
                    'BPC_syn': stat_values['all_BPC_syn#'] / (stat_values['all_syn_total'] - stat_values['all_Undetermined_subclass_syn#']),
                    'NGC_syn': stat_values['all_NGC_syn#'] / (stat_values['all_syn_total'] - stat_values['all_Undetermined_subclass_syn#']),
                    'Undetermined_subclass_syn': stat_values['all_Undetermined_subclass_syn#'] / stat_values['all_syn_total'],
        
        
                    'i_con': stat_values['all_i_con#'] / stat_values['all_con_total'],
                    'e_con': stat_values['all_e_con#'] / stat_values['all_con_total'],
                    'Undetermined_class_con': stat_values['all_Undetermined_class_con#'] / stat_values['all_con_total'],
                   
        
                    '23P_con': stat_values['all_23P_con#'] / stat_values['all_con_total'],
                    '4P_con': stat_values['all_4P_con#'] / stat_values['all_con_total'],
                    '5P-PT_con': stat_values['all_5P-PT_con#'] / stat_values['all_con_total'],
                    '5P-IT_con': stat_values['all_5P-IT_con#'] / stat_values['all_con_total'],
                    '5P-NP_con': stat_values['all_5P-NP_con#'] / stat_values['all_con_total'],
                    '6P_con': stat_values['all_6P_con#'] / stat_values['all_con_total'],
                    'BC_con': stat_values['all_BC_con#'] / stat_values['all_con_total'],
                    'MC_con': stat_values['all_MC_con#'] / stat_values['all_con_total'],
                    'BPC_con': stat_values['all_BPC_con#'] / stat_values['all_syn_total'],
                    'NGC_con': stat_values['all_NGC_con#'] / stat_values['all_con_total'],
                    'Undetermined_subclass_con': stat_values['all_Undetermined_subclass_con#'] / stat_values['all_con_total'],
                   
    }
    percentage.append(stat_percentage) 
    

synapse_table_values = pd.DataFrame(values)
synapse_table_percentage = pd.DataFrame(percentage)

#total_values = pd.DataFrame(synapse_table_values.sum(), columns=parts.columns, index=["Total"])
#stats_type.append(stat)   